<center><img src="http://i.imgur.com/sSaOozN.png" width="500"></center>

## Course: Computational Thinking for Governance Analytics

### Prof. José Manuel Magallanes, PhD 
* Visiting Professor of Computational Policy at Evans School of Public Policy and Governance, and eScience Institute Senior Data Science Fellow, University of Washington.
* Professor of Government and Political Methodology, Pontificia Universidad Católica del Perú. 

_____

# Data Preprocessing in Python: Data Integration and Reshaping

I will cover some important processes for DFs:
* Appending
* Reshaping
* Merging

# 1. Appending

As the name implies, this process binds DFs into one: one or more DFs will be put below or on top of another DF. The only requisites are that all of them share the same column names, and the same data types.

Let's visit this website: https://fundforpeace.org/what-we-do/country-risk-and-fragility-data/

There, you will find several excel files with the _Fragile States Index_ per year. Let's open the ones for years 2019-2021:

In [1]:
import pandas as pd

file2021="fsi-2021.xlsx"
file2020="fsi-2020.xlsx"
file2019="fsi-2019.xlsx"
    

# fetching the tables
fragil2021=pd.read_excel(file2021)
fragil2020=pd.read_excel(file2020)
fragil2019=pd.read_excel(file2019)

The goal is to append all of them. Let's see if they have the same column names with the help of **set**s:

In [2]:
A=['a','b','c']
B=['a','b','d','c']
C=['a','b','e','g']

set(A) & set(B) & set(C)

{'a', 'b'}

Then:

In [3]:
set(fragil2021.columns)&set(fragil2020)&set(fragil2019)

{'C1: Security Apparatus',
 'C2: Factionalized Elites',
 'C3: Group Grievance',
 'Country',
 'E1: Economy',
 'E2: Economic Inequality',
 'E3: Human Flight and Brain Drain',
 'P1: State Legitimacy',
 'P2: Public Services',
 'P3: Human Rights',
 'Rank',
 'S1: Demographic Pressures',
 'S2: Refugees and IDPs',
 'Total',
 'X1: External Intervention',
 'Year'}

In [4]:
sorted(list(set(fragil2021.columns)&set(fragil2020)&set(fragil2019)))

['C1: Security Apparatus',
 'C2: Factionalized Elites',
 'C3: Group Grievance',
 'Country',
 'E1: Economy',
 'E2: Economic Inequality',
 'E3: Human Flight and Brain Drain',
 'P1: State Legitimacy',
 'P2: Public Services',
 'P3: Human Rights',
 'Rank',
 'S1: Demographic Pressures',
 'S2: Refugees and IDPs',
 'Total',
 'X1: External Intervention',
 'Year']

Then:

In [5]:
common=sorted(list(set(fragil2021.columns)&set(fragil2020)&set(fragil2019)))
fragil2021=fragil2021.loc[:,common]
fragil2020=fragil2020.loc[:,common]
fragil2019=fragil2019.loc[:,common]

Let's check the data types with the help of **zip** and **set**:

In [6]:
A=['a','b','c']
B=['a','b','c']
C=['a','b','d']
list(zip(A,B,C))

[('a', 'a', 'a'), ('b', 'b', 'b'), ('c', 'c', 'd')]

Then, you detect where a group does not have all the elements equal:

In [7]:
[x for x in zip(A,B,C) if len(set(x))>1]

[('c', 'c', 'd')]

Applying that to our case:

In [8]:
theZips=zip(fragil2021.dtypes.index,fragil2021.dtypes,fragil2020.dtypes,fragil2019.dtypes)
[x for x in theZips if len(set(x))>2]

[('Year', dtype('int64'), dtype('<M8[ns]'), dtype('<M8[ns]'))]

The Year is not an integer in the DFs for 2020 and 2019, it is **date** type.

In [9]:
fragil2020['Year']=fragil2020['Year'].dt.year
fragil2019['Year']=fragil2019['Year'].dt.year

Now it should be fine:

In [10]:
theZips=zip(fragil2021.dtypes.index,fragil2021.dtypes,fragil2020.dtypes,fragil2019.dtypes)
[x for x in theZips if len(set(x))>2]

[]

Let's do the appending!

In [11]:
fragile=pd.concat([fragil2021,fragil2020,fragil2019])
fragile

,C1: Security Apparatus,C2: Factionalized Elites,C3: Group Grievance,Country,E1: Economy,E2: Economic Inequality,E3: Human Flight and Brain Drain,P1: State Legitimacy,P2: Public Services,P3: Human Rights,Rank,S1: Demographic Pressures,S2: Refugees and IDPs,Total,X1: External Intervention,Year
0,9.4,10.0,9.4,Yemen,9.8,7.7,6.7,9.9,9.8,9.7,1st,9.8,9.8,111.7,9.7,2021
1,9.3,10.0,8.3,Somalia,9.2,9.5,8.8,9.2,9.8,8.7,2nd,10.0,9.2,110.9,8.9,2021
2,9.6,9.9,9.7,Syria,9.6,7.1,8.2,10.0,9.6,9.7,3rd,7.6,9.7,110.7,10.0,2021
3,9.7,9.2,8.8,South Sudan,9.2,8.9,6.7,9.5,10.0,8.7,4th,9.5,9.9,109.4,9.3,2021
4,8.6,9.5,9.4,Congo Democratic Republic,8.5,8.5,6.8,9.2,9.8,9.2,5th,9.8,10.0,108.4,9.1,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,2.7,1.7,3.3,Australia,1.6,1.6,1.0,1.0,1.5,1.7,174th,1.2,1.7,19.7,0.7,2019
174,1.3,1.4,4.3,Denmark,1.6,1.2,1.9,0.9,0.9,1.7,175th,1.6,2.0,19.5,0.7,2019
175,1.1,1.0,3.3,Switzerland,1.9,1.8,1.7,0.7,1.0,1.4,176th,1.4,2.7,18.7,0.7,2019
176,2.1,1.1,3.3,Norway,1.9,1.0,1.3,0.6,0.8,0.9,177th,1.2,2.8,18.0,1.0,2019


You may notice that the column ordering does not look nice, let's try something:

In [12]:
fragile.set_index(['Country','Year','Total'],inplace=True)
fragile

,,,C1: Security Apparatus,C2: Factionalized Elites,C3: Group Grievance,E1: Economy,E2: Economic Inequality,E3: Human Flight and Brain Drain,P1: State Legitimacy,P2: Public Services,P3: Human Rights,Rank,S1: Demographic Pressures,S2: Refugees and IDPs,X1: External Intervention
Country,Year,Total,,,,,,,,,,,,,
Yemen,2021,111.7,9.4,10.0,9.4,9.8,7.7,6.7,9.9,9.8,9.7,1st,9.8,9.8,9.7
Somalia,2021,110.9,9.3,10.0,8.3,9.2,9.5,8.8,9.2,9.8,8.7,2nd,10.0,9.2,8.9
Syria,2021,110.7,9.6,9.9,9.7,9.6,7.1,8.2,10.0,9.6,9.7,3rd,7.6,9.7,10.0
South Sudan,2021,109.4,9.7,9.2,8.8,9.2,8.9,6.7,9.5,10.0,8.7,4th,9.5,9.9,9.3
Congo Democratic Republic,2021,108.4,8.6,9.5,9.4,8.5,8.5,6.8,9.2,9.8,9.2,5th,9.8,10.0,9.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Australia,2019,19.7,2.7,1.7,3.3,1.6,1.6,1.0,1.0,1.5,1.7,174th,1.2,1.7,0.7
Denmark,2019,19.5,1.3,1.4,4.3,1.6,1.2,1.9,0.9,0.9,1.7,175th,1.6,2.0,0.7
Switzerland,2019,18.7,1.1,1.0,3.3,1.9,1.8,1.7,0.7,1.0,1.4,176th,1.4,2.7,0.7


Delet _Rank_:

In [13]:
fragile.drop(columns='Rank',inplace=True)
fragile

,,,C1: Security Apparatus,C2: Factionalized Elites,C3: Group Grievance,E1: Economy,E2: Economic Inequality,E3: Human Flight and Brain Drain,P1: State Legitimacy,P2: Public Services,P3: Human Rights,S1: Demographic Pressures,S2: Refugees and IDPs,X1: External Intervention
Country,Year,Total,,,,,,,,,,,,
Yemen,2021,111.7,9.4,10.0,9.4,9.8,7.7,6.7,9.9,9.8,9.7,9.8,9.8,9.7
Somalia,2021,110.9,9.3,10.0,8.3,9.2,9.5,8.8,9.2,9.8,8.7,10.0,9.2,8.9
Syria,2021,110.7,9.6,9.9,9.7,9.6,7.1,8.2,10.0,9.6,9.7,7.6,9.7,10.0
South Sudan,2021,109.4,9.7,9.2,8.8,9.2,8.9,6.7,9.5,10.0,8.7,9.5,9.9,9.3
Congo Democratic Republic,2021,108.4,8.6,9.5,9.4,8.5,8.5,6.8,9.2,9.8,9.2,9.8,10.0,9.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Australia,2019,19.7,2.7,1.7,3.3,1.6,1.6,1.0,1.0,1.5,1.7,1.2,1.7,0.7
Denmark,2019,19.5,1.3,1.4,4.3,1.6,1.2,1.9,0.9,0.9,1.7,1.6,2.0,0.7
Switzerland,2019,18.7,1.1,1.0,3.3,1.9,1.8,1.7,0.7,1.0,1.4,1.4,2.7,0.7


Now put the row indexes back:

In [14]:
fragile.reset_index(inplace=True)
fragile

,Country,Year,Total,C1: Security Apparatus,C2: Factionalized Elites,C3: Group Grievance,E1: Economy,E2: Economic Inequality,E3: Human Flight and Brain Drain,P1: State Legitimacy,P2: Public Services,P3: Human Rights,S1: Demographic Pressures,S2: Refugees and IDPs,X1: External Intervention
0,Yemen,2021,111.7,9.4,10.0,9.4,9.8,7.7,6.7,9.9,9.8,9.7,9.8,9.8,9.7
1,Somalia,2021,110.9,9.3,10.0,8.3,9.2,9.5,8.8,9.2,9.8,8.7,10.0,9.2,8.9
2,Syria,2021,110.7,9.6,9.9,9.7,9.6,7.1,8.2,10.0,9.6,9.7,7.6,9.7,10.0
3,South Sudan,2021,109.4,9.7,9.2,8.8,9.2,8.9,6.7,9.5,10.0,8.7,9.5,9.9,9.3
4,Congo Democratic Republic,2021,108.4,8.6,9.5,9.4,8.5,8.5,6.8,9.2,9.8,9.2,9.8,10.0,9.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,Australia,2019,19.7,2.7,1.7,3.3,1.6,1.6,1.0,1.0,1.5,1.7,1.2,1.7,0.7
531,Denmark,2019,19.5,1.3,1.4,4.3,1.6,1.2,1.9,0.9,0.9,1.7,1.6,2.0,0.7
532,Switzerland,2019,18.7,1.1,1.0,3.3,1.9,1.8,1.7,0.7,1.0,1.4,1.4,2.7,0.7
533,Norway,2019,18.0,2.1,1.1,3.3,1.9,1.0,1.3,0.6,0.8,0.9,1.2,2.8,1.0


In [15]:
fragile.columns=fragile.columns.str.replace(':|\s',"",regex=True)
fragile

,Country,Year,Total,C1SecurityApparatus,C2FactionalizedElites,C3GroupGrievance,E1Economy,E2EconomicInequality,E3HumanFlightandBrainDrain,P1StateLegitimacy,P2PublicServices,P3HumanRights,S1DemographicPressures,S2RefugeesandIDPs,X1ExternalIntervention
0,Yemen,2021,111.7,9.4,10.0,9.4,9.8,7.7,6.7,9.9,9.8,9.7,9.8,9.8,9.7
1,Somalia,2021,110.9,9.3,10.0,8.3,9.2,9.5,8.8,9.2,9.8,8.7,10.0,9.2,8.9
2,Syria,2021,110.7,9.6,9.9,9.7,9.6,7.1,8.2,10.0,9.6,9.7,7.6,9.7,10.0
3,South Sudan,2021,109.4,9.7,9.2,8.8,9.2,8.9,6.7,9.5,10.0,8.7,9.5,9.9,9.3
4,Congo Democratic Republic,2021,108.4,8.6,9.5,9.4,8.5,8.5,6.8,9.2,9.8,9.2,9.8,10.0,9.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,Australia,2019,19.7,2.7,1.7,3.3,1.6,1.6,1.0,1.0,1.5,1.7,1.2,1.7,0.7
531,Denmark,2019,19.5,1.3,1.4,4.3,1.6,1.2,1.9,0.9,0.9,1.7,1.6,2.0,0.7
532,Switzerland,2019,18.7,1.1,1.0,3.3,1.9,1.8,1.7,0.7,1.0,1.4,1.4,2.7,0.7
533,Norway,2019,18.0,2.1,1.1,3.3,1.9,1.0,1.3,0.6,0.8,0.9,1.2,2.8,1.0


In [16]:
fragile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 535 entries, 0 to 534
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Country                     535 non-null    object 
 1   Year                        535 non-null    int64  
 2   Total                       535 non-null    float64
 3   C1SecurityApparatus         535 non-null    float64
 4   C2FactionalizedElites       535 non-null    float64
 5   C3GroupGrievance            535 non-null    float64
 6   E1Economy                   535 non-null    float64
 7   E2EconomicInequality        535 non-null    float64
 8   E3HumanFlightandBrainDrain  535 non-null    float64
 9   P1StateLegitimacy           535 non-null    float64
 10  P2PublicServices            535 non-null    float64
 11  P3HumanRights               535 non-null    float64
 12  S1DemographicPressures      535 non-null    float64
 13  S2RefugeesandIDPs           535 non

# 2. Reshaping

Let me keep some columns from the last DF:

In [17]:
fragileLong=fragile.iloc[:,:3]
fragileLong

,Country,Year,Total
0,Yemen,2021,111.7
1,Somalia,2021,110.9
2,Syria,2021,110.7
3,South Sudan,2021,109.4
4,Congo Democratic Republic,2021,108.4
...,...,...,...
530,Australia,2019,19.7
531,Denmark,2019,19.5
532,Switzerland,2019,18.7
533,Norway,2019,18.0


The unit of analysis is country. Generally, we are used to see the  unit of analysis once in a column, but here it is repeated for every year.

In [18]:
# wide
fragileWide=pd.pivot_table(fragileLong,
               values='Total',
               index=['Country'],
               columns=['Year']).\
            reset_index(drop=False).\
            rename_axis(index=None, columns=None)
fragileWide

,Country,2019,2020,2021
0,Afghanistan,105.0,102.901187,102.1
1,Albania,58.9,58.753811,59.0
2,Algeria,75.4,74.575183,73.6
3,Angola,87.8,87.320039,89.0
4,Antigua and Barbuda,54.4,52.062352,54.9
...,...,...,...,...
181,Venezuela,89.3,91.157591,92.6
182,Vietnam,66.1,63.885397,63.3
183,Yemen,113.5,112.438694,111.7
184,Zambia,85.7,84.474129,84.9


In [19]:
fragileWide[fragileWide.isna().any(axis=1)]

,Country,2019,2020,2021
27,Cabo Verde,NaN,NaN,64.2
31,Cape Verde,66.6,64.778171,NaN
45,Czech Republic,NaN,35.741616,NaN
46,Czechia,37.6,NaN,39.3
56,Eswatini,85.3,NaN,82.5
81,Israel,NaN,NaN,43.0
82,Israel and West Bank,76.5,75.123972,NaN
90,Kyrgyz Republic,76.2,73.929364,NaN
91,Kyrgyzstan,NaN,NaN,76.4
100,Macedonia,64.6,62.130864,NaN


In [20]:
changes={"Cabo Verde": "Cape Verde",
"Czechia":"Czech Republic",
"Swaziland":"Eswatini",
"Israel and West Bank":"Israel",
"Kyrgyzstan":"Kyrgyz Republic",
"North Macedonia":"Macedonia",
"Slovakia": "Slovak Republic"}
fragileLong.Country.replace(to_replace=changes,inplace=True)

In [21]:
# wide
fragileWide=pd.pivot_table(fragileLong,
               values='Total',
               index=['Country'],
               columns=['Year']).\
            reset_index(drop=False).\
            rename_axis(index=None, columns=None)
##
fragileWide[fragileWide.isna().any(axis=1)]

,Country,2019,2020,2021
124,Palestine,NaN,NaN,86.0


In [22]:
fragileWide[~fragileWide.isna().any(axis=1)]

,Country,2019,2020,2021
0,Afghanistan,105.0,102.901187,102.1
1,Albania,58.9,58.753811,59.0
2,Algeria,75.4,74.575183,73.6
3,Angola,87.8,87.320039,89.0
4,Antigua and Barbuda,54.4,52.062352,54.9
...,...,...,...,...
174,Venezuela,89.3,91.157591,92.6
175,Vietnam,66.1,63.885397,63.3
176,Yemen,113.5,112.438694,111.7
177,Zambia,85.7,84.474129,84.9


# 3. Merging

Integrating data sets needs the following considerations:

* Merging is done on two data frames.
* You need a column in each data frame with that share the same exact and unique values. The column names or titles need not be the same.
* The merged table shows by default the mutual coincidences; but you can also request the values not matched, which will help you detect possible extra cleaning.
* Pandas differentiates the **left** from the **right** data frames.

In [25]:
demo=pd.read_pickle("https://github.com/EvansDataScience/CTforGA_cleaning/raw/main/demoindex.pkl")
#and
fragile2021=fragile[fragile.Year==2021].drop(columns=['Year'])

In [27]:
fragile2021.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 179 entries, 0 to 178
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Country                     179 non-null    object 
 1   Total                       179 non-null    float64
 2   C1SecurityApparatus         179 non-null    float64
 3   C2FactionalizedElites       179 non-null    float64
 4   C3GroupGrievance            179 non-null    float64
 5   E1Economy                   179 non-null    float64
 6   E2EconomicInequality        179 non-null    float64
 7   E3HumanFlightandBrainDrain  179 non-null    float64
 8   P1StateLegitimacy           179 non-null    float64
 9   P2PublicServices            179 non-null    float64
 10  P3HumanRights               179 non-null    float64
 11  S1DemographicPressures      179 non-null    float64
 12  S2RefugeesandIDPs           179 non-null    float64
 13  X1ExternalIntervention      179 non

Remember the amount of rows of each DF:

In [28]:
demo.shape,fragile2021.shape

((167, 8), (179, 14))

The best scenario would be to create a merged DF with 167 rows, but the actual result is:

In [29]:
demo.merge(fragile2021).shape

(158, 21)

We are some countries far from the best scenario.

In [30]:
OnlyDemo=set(demo.Country)-set(fragile2021.Country)
OnlyDemo

{'Cape Verde',
 'Czech Republic',
 'Democratic Republic of the Congo',
 'East Timor',
 'Guinea-Bissau',
 'Hong Kong',
 'Ivory Coast',
 'Republic of the Congo',
 'Taiwan'}

In [31]:
OnlyFragile=set(fragile2021.Country)-set(demo.Country)
OnlyFragile

{'Antigua and Barbuda',
 'Bahamas',
 'Barbados',
 'Belize',
 'Brunei Darussalam',
 'Cabo Verde',
 'Congo Democratic Republic',
 'Congo Republic',
 "Cote d'Ivoire",
 'Czechia',
 'Grenada',
 'Guinea Bissau',
 'Maldives',
 'Micronesia',
 'Samoa',
 'Sao Tome and Principe',
 'Seychelles',
 'Solomon Islands',
 'Somalia',
 'South Sudan',
 'Timor-Leste'}

In [32]:
from thefuzz import process

[(process.extractOne(demo, OnlyFragile),demo) for demo in sorted(OnlyDemo)]

[(('Cabo Verde', 80), 'Cape Verde'),
 (('Congo Democratic Republic', 86), 'Czech Republic'),
 (('Congo Democratic Republic', 95), 'Democratic Republic of the Congo'),
 (('Timor-Leste', 82), 'East Timor'),
 (('Guinea Bissau', 100), 'Guinea-Bissau'),
 (('Congo Democratic Republic', 50), 'Hong Kong'),
 (("Cote d'Ivoire", 64), 'Ivory Coast'),
 (('Congo Republic', 86), 'Republic of the Congo'),
 (('Somalia', 46), 'Taiwan')]

In [33]:
[(process.extractOne(demo, OnlyFragile),demo) for demo in sorted(OnlyDemo) if process.extractOne(demo, OnlyFragile)[1]>90]

[(('Congo Democratic Republic', 95), 'Democratic Republic of the Congo'),
 (('Guinea Bissau', 100), 'Guinea-Bissau')]

In [34]:
{process.extractOne(demo, OnlyFragile)[0]:demo for demo in sorted(OnlyDemo) if process.extractOne(demo, OnlyFragile)[1]>90}

{'Congo Democratic Republic': 'Democratic Republic of the Congo',
 'Guinea Bissau': 'Guinea-Bissau'}

In [35]:
changesFragile1={process.extractOne(demo, OnlyFragile)[0]:demo for demo in sorted(OnlyDemo) if process.extractOne(demo, OnlyFragile)[1]>90}
fragile2021.Country.replace(changesFragile1,inplace=True)

In [36]:
OnlyDemo=set(demo.Country)-set(fragile2021.Country)
OnlyFragile=set(fragile2021.Country)-set(demo.Country)
[(process.extractOne(demo, OnlyFragile),demo) for demo in sorted(OnlyDemo)]

[(('Cabo Verde', 80), 'Cape Verde'),
 (('Congo Republic', 71), 'Czech Republic'),
 (('Timor-Leste', 82), 'East Timor'),
 (('Solomon Islands', 40), 'Hong Kong'),
 (("Cote d'Ivoire", 64), 'Ivory Coast'),
 (('Congo Republic', 86), 'Republic of the Congo'),
 (('Somalia', 46), 'Taiwan')]

In [37]:
changesFragile2={process.extractOne(demo, OnlyFragile)[0]:demo for demo in sorted(OnlyDemo) if process.extractOne(demo, OnlyFragile)[1]>79}
fragile2021.Country.replace(changesFragile2,inplace=True)

In [38]:
OnlyDemo=set(demo.Country)-set(fragile2021.Country)
OnlyFragile=set(fragile2021.Country)-set(demo.Country)
[(process.extractOne(demo, OnlyFragile),demo) for demo in sorted(OnlyDemo)]

[(('Czechia', 64), 'Czech Republic'),
 (('Solomon Islands', 40), 'Hong Kong'),
 (("Cote d'Ivoire", 64), 'Ivory Coast'),
 (('Somalia', 46), 'Taiwan')]

In [39]:
changesFragile3={process.extractOne(demo, OnlyFragile)[0]:demo for demo in sorted(OnlyDemo) if process.extractOne(demo, OnlyFragile)[1]>60}
fragile2021.Country.replace(changesFragile3,inplace=True)

In [40]:
demo.merge(fragile2021).shape

(165, 21)

Then:

In [41]:
demo_fragile=demo.merge(fragile2021)

In [42]:
demo_fragile.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 165 entries, 0 to 164
Data columns (total 21 columns):
 #   Column                        Non-Null Count  Dtype   
---  ------                        --------------  -----   
 0   Country                       165 non-null    object  
 1   Regimetype                    165 non-null    category
 2   Overallscore                  165 non-null    float64 
 3   Electoralprocessandpluralism  165 non-null    float64 
 4   Functioningofgovernment       165 non-null    float64 
 5   Politicalparticipation        165 non-null    float64 
 6   Politicalculture              165 non-null    float64 
 7   Civilliberties                165 non-null    float64 
 8   Total                         165 non-null    float64 
 9   C1SecurityApparatus           165 non-null    float64 
 10  C2FactionalizedElites         165 non-null    float64 
 11  C3GroupGrievance              165 non-null    float64 
 12  E1Economy                     165 non-null    floa

In [43]:
demo_fragile.describe()

,Overallscore,Electoralprocessandpluralism,Functioningofgovernment,Politicalparticipation,Politicalculture,Civilliberties,Total,C1SecurityApparatus,C2FactionalizedElites,C3GroupGrievance,E1Economy,E2EconomicInequality,E3HumanFlightandBrainDrain,P1StateLegitimacy,P2PublicServices,P3HumanRights,S1DemographicPressures,S2RefugeesandIDPs,X1ExternalIntervention
count,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000
mean,5.256970,5.619091,4.615758,5.377091,5.350909,5.326242,66.706061,5.293939,6.600606,5.758182,5.778182,5.254545,5.211515,5.816970,5.718788,5.446667,5.900606,4.801818,5.124242
std,2.309758,3.825207,2.566877,1.966140,1.826606,2.697269,23.869811,2.282401,2.436273,2.323556,2.079900,2.177167,2.014048,2.846363,2.526653,2.612319,2.251680,2.353525,2.501881
min,0.320000,0.000000,0.000000,0.000000,1.250000,0.000000,16.200000,0.500000,1.000000,0.500000,1.200000,1.000000,0.500000,0.500000,1.200000,0.500000,1.400000,0.700000,0.500000
25%,3.200000,1.420000,2.500000,3.890000,3.750000,3.240000,50.100000,3.500000,5.100000,4.000000,4.400000,3.600000,3.800000,3.700000,3.700000,3.400000,4.100000,2.800000,3.400000
50%,5.610000,7.000000,5.000000,5.560000,5.000000,5.590000,70.400000,5.600000,7.300000,5.800000,5.900000,5.200000,5.500000,6.400000,5.400000,6.000000,5.700000,4.300000,5.400000
75%,7.060000,9.170000,6.430000,6.670000,6.250000,7.650000,84.100000,6.900000,8.400000,7.500000,7.100000,7.100000,6.800000,8.200000,8.000000,7.400000,8.000000,6.600000,6.900000
max,9.750000,10.000000,9.640000,10.000000,10.000000,9.710000,111.700000,10.000000,10.000000,9.900000,9.800000,9.600000,9.000000,10.000000,10.000000,9.800000,9.800000,10.000000,10.000000


In [44]:
demo_fragile[['Total']]

,Total
0,16.6
1,18.4
2,16.2
3,21.4
4,18.0
...,...
160,107.0
161,108.4
162,90.0
163,93.8


In [46]:
from sklearn import preprocessing

scaler = preprocessing.MinMaxScaler(feature_range=(0, 10))

df = scaler.fit_transform(demo_fragile[['Total']])
scaled_df = pd.DataFrame(df, columns=['Total'])
scaled_df.describe()

,Total
count,165.000000
mean,5.288593
std,2.499457
min,0.000000
25%,3.549738
50%,5.675393
75%,7.109948
max,10.000000


In [47]:
demo_fragile['Total']=scaled_df.Total

____


# <font color="red">Exporting file</font>

The current _demo_fragile_ data frame is clean and formatted.It is time to send it to a format that will keep all our work for future use:

#### For future use in Python:

In [48]:
demo_fragile.to_pickle("demo_fragile.pkl")
# you will need: DF=pd.read_pickle("demo_fragile.pkl")
# or:
# from urllib.request import urlopen
# DF=pd.read_pickle(urlopen("https://..../demo_fragile.pkl"),compression=None)

#### For future  use in R:

In [49]:
#try the following before starting Python:
#export LD_LIBRARY_PATH="$(python -m rpy2.situation LD_LIBRARY_PATH)":${LD_LIBRARY_PATH}

from rpy2.robjects import pandas2ri
pandas2ri.activate()

from rpy2.robjects.packages import importr

base = importr('base')
base.saveRDS(demo_fragile,file="demo_fragile.RDS")


#In R, you call it with: DF = readRDS("demo_fragile.RDS")
#or, if iyou read from cloud: DF = readRDS(url("https://..../demo_fragile.RDS")

<rpy2.rinterface_lib.sexp.NULLType object at 0x7f9776186a50> [RTYPES.NILSXP]